In [3]:
# spot_riskAssessment_v001

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X' #add the API
SINCE_DATE = int(datetime(2023, 1, 1).timestamp())  # Jan 1, 2023
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/spot_cvd_sum',
    'https://api.glassnode.com/v1/metrics/market/spot_volume_daily_sum'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_momentum_rsi(df, column='price_usd_close', window=90, normalize=True):
    # Calculate price changes
    price_change = df[column].diff()

    # Calculate gains and losses
    gains = price_change.where(price_change > 0, 0)
    losses = -price_change.where(price_change < 0, 0)

    # Calculate average gains and losses
    avg_gains = gains.rolling(window=window, min_periods=1).mean()
    avg_losses = losses.rolling(window=window, min_periods=1).mean()

    # Calculate relative strength
    relative_strength = avg_gains / avg_losses

    # Calculate RSI
    rsi = 100 - (100 / (1 + relative_strength))

    if normalize:
        # Normalize RSI to -1 to +1 range
        rsi_min = rsi.rolling(window=window, min_periods=1).min()
        rsi_max = rsi.rolling(window=window, min_periods=1).max()
        normalized_momentum = 2 * (rsi - rsi_min) / (rsi_max - rsi_min) - 1
        return normalized_momentum
    else:
        return rsi

def calculate_spot_cvd_bias(df, column='spot_cvd_sum', window_sum=7, window_norm=30, normalize=True):
    # Calculate rolling sum
    rolling_sum = df[column].rolling(window=window_sum).sum()
    
    if normalize:
        # Normalize to -1 to +1 range
        rolling_min = rolling_sum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = rolling_sum.rolling(window=window_norm, min_periods=1).max()
        normalized_bias = 2 * (rolling_sum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_bias
    else:
        return rolling_sum

# Apply the functions to our merged_df
merged_df['Price Momentum'] = calculate_momentum_rsi(merged_df, column='price_usd_close', window=90, normalize=True)
merged_df['Spot CVD Bias'] = calculate_spot_cvd_bias(merged_df, column='spot_cvd_sum', window_sum=7, window_norm=30, normalize=True)

In [4]:
merged_df.tail()

,price_usd_close,spot_cvd_sum,spot_volume_daily_sum,Price Momentum,Spot CVD Bias
t,,,,,
2024-08-31,58971.654336,-2.149215e+07,2.178212e+09,-0.430872,-0.389954
2024-09-01,57319.948544,-2.361100e+08,5.535664e+09,-0.648162,-0.615546
2024-09-02,59099.379977,9.461362e+06,6.223415e+09,-0.651026,-0.431569
2024-09-03,57469.422699,-1.274605e+08,6.259370e+09,-0.813136,-0.290310
2024-09-04,57982.587216,1.339837e+07,9.142887e+09,-0.744197,-0.148374
